In [145]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [146]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).text

### Parse wiki doc

In [147]:
soup = BeautifulSoup(html,'html.parser')

In [148]:
df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
rows = soup.findAll('tr')
i=-0
for tr in rows:
    cells =tr.findAll('td')
    c1= [c.text for c in cells]
    if(len(c1) != 0 and c1[0].startswith('M') and ~c1[1].find("Not assigned")>=0):    
        df.loc[i] = [c1[0],c1[1],c1[2] ]
        i=i+1
       

In [149]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [150]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [151]:
df.shape

(212, 3)

#### Replace unwanted text '\n' at the end of Neighborhood

In [152]:
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')

In [153]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### If borough is there but Neighborhood is not assigned then borough and neighborhood should be same

In [154]:
df[df['Borough'].str.contains('Park')]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [155]:
#check
df.loc[df['Neighborhood']=='Not assigned','PostalCode']


6    M7A
Name: PostalCode, dtype: object

In [156]:
df.loc[df['Neighborhood']=='Not assigned','Borough']

6    Queen's Park
Name: Borough, dtype: object

In [157]:
#Assign all neighborhoods to borough whose neighborhood are not assigned
df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df.loc[df['Neighborhood']=='Not assigned','Borough'] 

In [158]:
df[df['Borough'].str.contains('Park')]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Queen's Park


#### Merger more than one same neighborhood into a single row sepearted by columns

In [159]:
# Example row
df[df['PostalCode'].str.contains('M5A')]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park


In [160]:
#Merge all rows and create dataframe
dfNew = df.groupby(['PostalCode','Borough'],sort=False).agg(lambda col: ', '.join(col))
dfNew.reset_index(inplace=True)
dfNew[dfNew['PostalCode'].str.contains('M5A')]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [161]:
dfNew.shape

(103, 3)

## Merge Lat and Long

Geospatial_Coordiantes.csv file contains lat and long of canada according to postal code

In [163]:
cord = pd.read_csv("Geospatial_Coordinates.csv")
cord.columns = ['PostalCode','Latitude','Longitude']
cord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [165]:
#search critieria
#cord[cord['PostalCode'].str.contains('M5G')]

for row in dfNew.itertuples(index=True, name='PostalCode'):
    print(row[1])

#dfNew.loc[cord['M2H']


M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [69]:
dfNew[dfNew['PostalCode'].str.contains('M5G')].Latitude

24   NaN
Name: Latitude, dtype: float64

In [63]:
#retrieve and update the lat and long
def get_lat_long(row):
    row[row['PostalCode']]['Latitude'] = cord[cord['PostalCode'].str.contains(row['PostalCode'])]['Latitude']
    

In [64]:
dfNew.apply(get_lat_long, axis=1)
dfNew.head()

KeyError: ('M3A', 'occurred at index 0')